# Install Dependencies

In [1]:
%%capture
!pip install llama-index-core pymupdf llama-index chromadb sentence-transformers pydantic llama-index-llms-openai llama-index-embeddings-huggingface llama-index-vector-stores-chroma
!pip install llama-index-readers-file
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-llms-azure-openai

In [22]:
# สร้าง retriever
import chromadb
import os
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import PromptTemplate

# นำเข้าไฟล์ .env 
from dotenv import load_dotenv
load_dotenv()

PDF_DATA_PATH="../data/personal_data_protection_policy.pdf"
EMBEDDING_MODEL="BAAI/bge-base-en-v1.5"
CHUNK_SIZE=1000
CHUNK_OVERLAP=250
OPENAI_TEMP=0
CHROMA_PERSISTENCE_PATH="./chroma_db"
CHROMA_COLLECTION_NAME="test4543ss332"
RETRIEVER_TOP_K=5

# Setup LLM & Embeddings

In [16]:
embed_model = OpenAIEmbedding(model="text-embedding-3-large")

embeddings = embed_model.get_text_embedding(
    "Open AI new Embeddings models is great."
)
print(embeddings)

[-0.011500772088766098, 0.02457442320883274, -0.01760469563305378, -0.017763426527380943, 0.029841400682926178, -0.01646471954882145, -0.012878844514489174, 0.04998578876256943, 0.007914897985756397, 0.016277126967906952, 0.027864480391144753, -0.02102462388575077, 0.001592719811014831, -0.00906930398195982, -0.012337716296315193, -0.0049531240947544575, 0.0152670219540596, 0.01217898540198803, -0.009206390008330345, -0.02884572558104992, -0.04037535935640335, -0.03191933408379555, -0.025786548852920532, -0.04865822196006775, 0.028802435845136642, -0.011443051509559155, 0.0015223731752485037, 0.02376633882522583, -0.014805260114371777, 0.038730330765247345, 0.008456025272607803, -0.03454560786485672, 0.0031042704358696938, 0.02531035616993904, -0.0029545584693551064, 0.004513006657361984, 0.045425884425640106, 0.025959709659218788, -0.043232511729002, 7.801260653650388e-05, 0.030620625242590904, -0.022727372124791145, -0.008225144818425179, -0.003465022426098585, 0.0020436597988009453,

In [17]:
LLM = OpenAI(model="gpt-4o")

# ลองใช้งาน LLM
LLM.complete("What is the capital of Thailand?")

CompletionResponse(text='The capital of Thailand is Bangkok.', additional_kwargs={}, raw=ChatCompletion(id='chatcmpl-A5aY3Ob5EOcm3JttdRTfQoOgaM9eX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Thailand is Bangkok.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725895495, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=7, prompt_tokens=14, total_tokens=21)), logprobs=None, delta=None)

# Setup LlamaIndex Embeddings and Chunking

In [24]:
splitter = TokenTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

Settings.node_parser = splitter

Settings.embed_model = embed_model

Settings.chunk_size=CHUNK_SIZE
Settings.chunk_overlap=CHUNK_OVERLAP

Settings.llm = LLM

# Read PDFs

In [10]:
from llama_index.readers.file.docs import PDFReader

reader = PDFReader()

documents = reader.load_data(PDF_DATA_PATH)

In [13]:
# check documents
docs = [doc.text for doc in documents]
print("Document's length: ", len(docs))
print("\n".join(docs))

print(documents)

Document's length:  9
 
 
  
 
 
 
  
 
 
บริษัท  ซีพีเอฟ เทรดดิ้ง จํากัด  
 
 
 
นโยบายว่าด้วยการค ุ้มครองข้อม ูลส่วนบ ุคคล  
 
 
 
  
 
 
 
 
 
 
    
1 นโยบายว่าด้วยการค ุ้มครองข้อม ูลส่วนบ ุคคล 
 
 บริษัท ซีพีเอฟ เทรดดิ้ง จํากัด 
1. วัตถุประสงค์  
บริษัท ซีพีเอฟ เทรดดิ ้ง จํากัด  เคารพและให้ความสําคัญในการคุ้มครองข้อมูลส ่วนบุคคลของบุคลากร  ลูกค้า คู ่
ค้าธุรก ิจ และพันธมิตรทางธุรก ิจของบริษัทฯ และบริษัทย ่อย โดยบริษัทฯ และบริษัทย ่อยมีความมุ ่งมั่นที่จะ
ปกป้ อง ข้อมูลส ่วนบุคคลจากการถูกนําไปใช้ในทางที่ ไม่ถูกต้องตามกฎหมาย และรักษาข้อมูล ดังกล ่าวให้
ปลอดภัยตามมาตรฐานสากล ซึ่งเป ็ นที่ยอมรับ บริษัทฯ จึงได้ จัดทํานโยบาย ฯ ฉบับนี ้ โดยมีวัตถุประสงค์ ดังนี ้ 
1.1 เพื่อให้การทําธุรกรรมก ับบริษัทฯ มีความมั ่นคงปลอดภัย น ่าเชื่อถือ และสร้างความมั ่นใจให้แก ่เจ้าของ
ข้อมูลส ่วนบุคคล  
1.2 เพื่อป้องก ันความเสียหายที่เก ิดจากการนําข้อมูลส ่วนบุคคลไปแสวงหาประโยชน์โดยทุจริตหรือนําไปใช้
ในทางที่ ไม่ถูกต้องตามกฎหมาย  
1.3 เพื่อปฏิบัติตาม กฎหมาย ว่าด้วยการ คุ้มครองข้อมูลส ่วนบุคคล  
2. ขอบเขต  


# Setup Chroma Vector Store and Ingest Documents

In [34]:
def generate_retriever(documents) -> VectorStoreIndex:
    db = chromadb.Client()
    
    # [OPTIONAL] delete if existing
    try:
        db.delete_collection(CHROMA_COLLECTION_NAME)
    except ValueError:
        print("[ChromaDB] Failed to delete collection.")
        
    chroma_collection = db.create_collection(CHROMA_COLLECTION_NAME)
    
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # StorageContext is not ded like the other one
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    nodes = splitter.get_nodes_from_documents(documents)
    
    index = VectorStoreIndex(nodes, storage_context=storage_context, show_progress=True)

    return index

vector_store_index = generate_retriever(documents) 
retriever = vector_store_index.as_retriever(similarity_top_k=RETRIEVER_TOP_K)

Generating embeddings:   0%|          | 0/17 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 17/17 [00:01<00:00,  9.94it/s]


In [35]:
# Sanity check for docs in the vDB
res = retriever.retrieve("การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?")

In [36]:
print(f"Retrieve {len(res)} chunks")

Retrieve 5 chunks


# Testing our RAG workflow

In [37]:
query_engine = vector_store_index.as_query_engine()

In [38]:
user_query = "การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?"
response = query_engine.query(user_query)

In [39]:
print(response)

การเก็บรวบรวมข้อมูลส่วนบุคคลมีแนวทางดังนี้:

1. ข้อมูลส่วนบุคคลจะถูกเก็บรวบรวมตามวัตถุประสงค์ที่กำหนดไว้และจะถูกทำลายหรือลบเมื่อพ้นระยะเวลาการใช้งานตามวัตถุประสงค์นั้น หรือเมื่อเกินความจำเป็น
2. ข้อมูลส่วนบุคคลจะถูกทำลายหรือลบเมื่อเจ้าของข้อมูลร้องขอหรือถอนความยินยอม เว้นแต่จะมีข้อกำหนดทางกฎหมายที่อนุญาตให้เก็บรักษาข้อมูลต่อไป
3. การลบหรือทำลายข้อมูลส่วนบุคคลต้องเป็นวิธีการที่ปลอดภัยและไม่ทำให้ข้อมูลรั่วไหล
4. ในกรณีที่มีการว่าจ้างบุคคลภายนอกเพื่อประมวลผลข้อมูลส่วนบุคคล จะต้องดำเนินการตามคำสั่งหรือในนามของบริษัทและต้องปฏิบัติตามกฎหมายที่เกี่ยวข้อง


## Using the prompt-template

ทดลองใช้  Template เพื่อให้เราได้คำตอบในรูปแบบที่เราต้องการ

In [41]:
QUERY_PROMPT_TEMPLATE = """
# Task
From given document answer the following question.

# Instructions
- Answer the following question based on the given document.
- You need to cite the source of the answer from the document.

# Document:
{document}

# Question:
{question}

# Answer:
"""

In [48]:
def get_answer_from_rag(
    retriever: BaseRetriever,
    llm: OpenAI,
    qa_prompt: PromptTemplate,
    question: str,
) -> str:
    # Retrieve the chunks
    chunks = retriever.retrieve(question)
    response = llm.complete(
        qa_prompt.format(
            document=chunks,
            question=question,
        )
    )
    
    return response


In [49]:
answer = get_answer_from_rag(
    retriever=query_engine,
    llm=LLM,
    qa_prompt=QUERY_PROMPT_TEMPLATE,
    question="การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?",
)

In [50]:
print("Answer: ", answer)

Answer:  การเก็บรวบรวมข้อมูลส่วนบุคคลมีแนวทางดังนี้:

1. เมื่อพ้นระยะเวลาการใช้งานตามวัตถุประสงค์ในการเก็บรวบรวมข้อมูลส่วนบุคคล
2. เมื่อเกินความจำเป็น
3. เมื่อเจ้าของข้อมูลส่วนบุคคลร้องขอ
4. เมื่อเจ้าของข้อมูลได้ถอนความยินยอม

เว้นแต่หน่วยงานหรือผู้รับผิดชอบจะสามารถเก็บรักษาข้อมูลส่วนบุคคลไว้ต่อไปตามที่กฎหมายแต่ละประเทศกำหนด

(ที่มา: personal_data_protection_policy.pdf, หน้า 8)
